In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing
import statsmodels.formula.api as smf
from pandas import read_csv
import pandas as pd
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

%matplotlib inline

train = pd.read_csv('dataset/sales_train.csv')
test = pd.read_csv('dataset/test.csv')
shops = pd.read_csv('dataset/shops.csv')
items = pd.read_csv('dataset/items.csv')
items_categories = pd.read_csv('dataset/item_categories.csv')
#train.describe()

FileNotFoundError: File b'dataset/sales_train.csv' does not exist

In [2]:
def preprocess_date(data):
    data['month'], data['date'], data['year'] = data['date'].str.split('.').str
    data[['month']] = data[['month']].astype(int)
    data[['date']] = data[['date']].astype(int)
    data[['year']] = data[['year']].astype(int)
    return data
    
print(preprocess_date(train).head())

   date  date_block_num  shop_id  item_id  item_price  item_cnt_day  month  \
0     1               0       59    22154      999.00           1.0      2   
1     1               0       25     2552      899.00           1.0      3   
2     1               0       25     2552      899.00          -1.0      5   
3     1               0       25     2554     1709.05           1.0      6   
4     1               0       25     2555     1099.00           1.0     15   

   year  
0  2013  
1  2013  
2  2013  
3  2013  
4  2013  


In [3]:
gp = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': ['sum']})
X = np.array(list(map(list, gp.index.values)))
Y_train = gp.values
test['date_block_num'] = train['date_block_num'].max() + 1
X_test = test[['date_block_num', 'shop_id', 'item_id']].values

oh0 = OneHotEncoder().fit(X[:,0].reshape(-1, 1))
x0 = oh0.transform(X[:,0].reshape(-1, 1))
oh1 = OneHotEncoder().fit(X[:,1].reshape(-1, 1))
x1 = oh1.transform(X[:,1].reshape(-1, 1))
x1_t = oh1.transform(X_test[:,1].reshape(-1, 1))
X_train = np.concatenate((X[:, :1], x1.toarray(), X[:, 2:]), axis=1)
X_test = np.concatenate((X_test[:, :1], x1_t.toarray(), X_test[:, 2:]), axis=1)



/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [4]:
#linear regression
linreg= LinearRegression().fit(X_train, Y_train.ravel())
Y_test = linreg.predict(X_test)
submission = pd.read_csv('dataset/sample_submission.csv')
submission['item_cnt_month'] = Y_test
submission.to_csv('submission.csv', index=False)

In [9]:
#knn
best_score = 99
best_k = -1
'''
for k in range(7, 12):
    knn = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(knn, X_train, Y_train.ravel(), cv=5)
    score = np.mean(scores)
    if score < best_score:
        best_score = score
        best_k = k
print(best_score, k)

'''

knn = KNeighborsRegressor(n_neighbors=11)
knn.fit(X_train, Y_train.ravel())
Y_test = knn.predict(X_test)
submission = pd.read_csv('dataset/sample_submission.csv')
submission['item_cnt_month'] = Y_test
submission.to_csv('submission.csv', index=False)